In [1]:
from typing import Optional, List

from sqlmodel import (
    Field,
    Relationship,
    SQLModel,
    create_engine,
    Session,
    delete,
)

import sqlalchemy

class Contact(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    invoicing_contact_of: List["Client"] = Relationship(
        back_populates="invoicing_contact",
        sa_relationship_kwargs={"lazy": "subquery", "passive_deletes": "all"},
    )


class Client(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    invoicing_contact_id: int = Field(default=None, foreign_key="contact.id")
    invoicing_contact: Contact = Relationship(
        back_populates="invoicing_contact_of",
        sa_relationship_kwargs={"lazy": "subquery"},
    )


engine = create_engine("sqlite:///", echo=True)
sqlalchemy.event.listen(engine, "connect", lambda c, _: c.execute("PRAGMA foreign_keys = ON"))

SQLModel.metadata.drop_all(engine)
SQLModel.metadata.create_all(engine)



2023-02-09 15:37:07,196 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-02-09 15:37:07,197 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("contact")
2023-02-09 15:37:07,198 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-09 15:37:07,198 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("contact")
2023-02-09 15:37:07,198 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-09 15:37:07,199 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("client")
2023-02-09 15:37:07,200 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-09 15:37:07,201 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("client")
2023-02-09 15:37:07,202 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-09 15:37:07,203 INFO sqlalchemy.engine.Engine COMMIT
2023-02-09 15:37:07,203 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-02-09 15:37:07,204 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("contact")
2023-02-09 15:37:07,205 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-09 15:37:07,205 

In [2]:
with Session(engine) as session:
    contact = Contact()
    client = Client(invoicing_contact=contact)

    session.add(client)
    session.commit()
    session.refresh(client)

    session.exec(delete(Contact).where(Contact.id == 1))
    session.commit()

2023-02-09 15:37:07,307 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-02-09 15:37:07,310 INFO sqlalchemy.engine.Engine INSERT INTO contact DEFAULT VALUES
2023-02-09 15:37:07,311 INFO sqlalchemy.engine.Engine [generated in 0.00093s] ()
2023-02-09 15:37:07,314 INFO sqlalchemy.engine.Engine INSERT INTO client (invoicing_contact_id) VALUES (?)
2023-02-09 15:37:07,315 INFO sqlalchemy.engine.Engine [generated in 0.00152s] (1,)
2023-02-09 15:37:07,318 INFO sqlalchemy.engine.Engine COMMIT
2023-02-09 15:37:07,320 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-02-09 15:37:07,324 INFO sqlalchemy.engine.Engine SELECT client.id, client.invoicing_contact_id 
FROM client 
WHERE client.id = ?
2023-02-09 15:37:07,326 INFO sqlalchemy.engine.Engine [generated in 0.00150s] (1,)
2023-02-09 15:37:07,329 INFO sqlalchemy.engine.Engine SELECT contact.id AS contact_id 
FROM contact 
WHERE contact.id = ?
2023-02-09 15:37:07,330 INFO sqlalchemy.engine.Engine [generated in 0.00087s] (1,)
2023-02-09 15:3

IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: DELETE FROM contact WHERE contact.id = ?]
[parameters: (1,)]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

In [ ]:
with Session(engine) as session:
    contact = Contact()
    client = Client(invoicing_contact=contact)

    session.add(client)
    session.commit()
    session.refresh(client)

    session.exec(delete(Client).where(Client.id == 1))
    session.commit()